<a href="https://colab.research.google.com/github/akshaya-devaraj/aimlproject/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install keras-preprocessing

In [7]:
!pip install numpy pandas matplotlib tensorflow keras scikit-learn

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Set the path to the dataset
base_dir = '/content/drive/MyDrive/project'
images_dir = os.path.join(base_dir, 'images')  # Assuming images are in a subdirectory

In [9]:
# Load the image labels and class mappings
labels_df = pd.read_csv(os.path.join(base_dir, 'images.txt'), header=None, delim_whitespace=True)
labels_df.columns = ['image_id', 'image_name']
labels_df['class_id'] = pd.read_csv(os.path.join(base_dir, 'image_class_labels.txt'), header=None, delim_whitespace=True)[1].values

# Load classes
class_mapping = pd.read_csv(os.path.join(base_dir, 'classes.txt'), header=None, delim_whitespace=True)
class_mapping.columns = ['class_id', 'class_name']

# Display the classes
print(class_mapping.head())

   class_id                  class_name
0         1  001.Black_footed_Albatross
1         2        002.Laysan_Albatross
2         3         003.Sooty_Albatross
3         4       004.Groove_billed_Ani
4         5          005.Crested_Auklet


In [10]:
# Define image parameters
img_height, img_width = 150, 150
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
# For validation set, only rescaling is applied
valid_datagen = ImageDataGenerator(rescale=1.0/255)

# Create a function to load images and labels
def load_images_and_labels(df):
    images = []
    labels = []

    for _, row in df.iterrows():
        img_path = os.path.join(images_dir, row['image_name'])
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img) / 255.0  # Normalize
        images.append(img_array)
        labels.append(row['class_id'] - 1)  # Adjust for zero-based indexing

    return np.array(images), np.array(labels)

In [12]:
# Load all images and labels
images, labels = load_images_and_labels(labels_df)

# Split into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_mapping), activation='softmax'))  # Number of classes
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, batch_size=batch_size)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/project/images/001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg'

In [13]:
# Plot training & validation accuracy and loss
def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    plt.show()

plot_history(history)

NameError: name 'history' is not defined

In [14]:
# Evaluate the model on validation set
val_loss, val_accuracy = model.evaluate(X_valid, y_valid)
print(f"Validation Accuracy: {val_accuracy:.2f}")

NameError: name 'model' is not defined

In [15]:
# Function to predict species
def predict_bird_species(img_path):
    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = model.predict(img_array)
    predicted_class = class_mapping.iloc[np.argmax(predictions)]['class_name']
    return predicted_class

In [16]:
# Example prediction
print(predict_bird_species('/content/boris-smokrovic-DPXytK8Z59Y-unsplash.jpg'))  # Update with your image path

NameError: name 'model' is not defined